## IMDB Movie reviews

In [2]:
import numpy as np
import pandas as pd

In [28]:
df = pd.read_csv('labeledTrainData.tsv', sep = '\t')
df

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [29]:
df.review[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [30]:
# <br /> to space
df['review'] = df.review.str.replace('<br />', ' ')
df.review[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.  Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.  The actual feature film bit when it finally starts is only on for 2

In [31]:
# Remove numbers and symbols
import re

df['review'] = df.review.apply(lambda x: re.sub('[^A-Za-z]', ' ', x))
df.review[0]

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

#### Dividing data as train / test and text transition

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, stratify = df.sentiment, random_state = 2021
)

((18750,), (6250,), (18750,), (6250,))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words = 'english', ngram_range = (1, 2))
cv.fit(X_train)
X_train_cv = cv.transform(X_train)
X_test_cv = cv.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((18750, 1375412), (6250, 1375412))

- KNeighborsClassifier

In [9]:
from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier()
knc.fit(X_train_cv, y_train)

KNeighborsClassifier()

In [11]:
from sklearn.metrics import accuracy_score

pred = knc.predict(X_test_cv)
accuracy_score(y_test, pred)

0.5512

- LogisticRegression

In [12]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter = 300)
lr.fit(X_train_cv, y_train)

LogisticRegression(max_iter=300)

In [13]:
pred = lr.predict(X_test_cv)
accuracy_score(y_test, pred)

0.88672

- TfidfVectorizer

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(stop_words = 'english', ngram_range = (1, 2))
tv.fit(X_train)
X_train_tv = tv.transform(X_train)
X_test_tv = tv.transform(X_test)

In [15]:
lr = LogisticRegression(max_iter = 300)
lr.fit(X_train_tv, y_train)
pred = lr.predict(X_test_tv)
accuracy_score(y_test, pred)

0.87776

### Model 저장하고 불러오기

In [16]:
import joblib

In [17]:
joblib.dump(tv, 'model/imdb_tv.pkl')
joblib.dump(lr, 'model/imdb_lr.pkl')

['model/imdb_lr.pkl']

In [18]:
del tv
del lr

In [19]:
new_tv = joblib.load('model/imdb_tv.pkl')
new_lr = joblib.load('model/imdb_lr.pkl')

In [20]:
new_X_test_tv = new_tv.transform(X_test)
new_pred = new_lr.predict(new_X_test_tv)
accuracy_score(y_test, new_pred)

0.87776

### Hyper Parameter tuning through Pipeline and GridSearchCV

In [21]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tv', TfidfVectorizer(stop_words = 'english', ngram_range = (1, 2))),
    ('lr', LogisticRegression())
])

params = {
    'tv__max_df' : [100, 500],
    'lr__C' : [1, 10],
}

In [23]:
from sklearn.model_selection import GridSearchCV

pipe_grid = GridSearchCV(pipeline, param_grid = params, cv = 3, scoring = 'accuracy', verbose = 1, n_jobs= -1)
pipe_grid.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tv',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('lr', LogisticRegression())]),
             n_jobs=-1, param_grid={'lr__C': [1, 10], 'tv__max_df': [100, 500]},
             scoring='accuracy', verbose=1)

In [24]:
pipe_grid.best_score_

0.88128

In [25]:
pipe_grid.best_params_

{'lr__C': 10, 'tv__max_df': 500}

In [26]:
pred = pipe_grid.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.88096

In [27]:
joblib.dump(pipe_grid, 'model/imdb_pipe.pkl')

['model/imdb_pipe.pkl']